**PCS5024 - Aprendizado Estatístico**

**Name**: Rodrigo da Silva Cunha

**NUSP**: 7631302

# Building Classifiers for Adult Dataset

In this notebook we'll explore the Adult Dataset, available for download at the [UCI Repository](https://archive.ics.uci.edu/ml/datasets/adult), and build a few classifiers to predict the class variable ">50k", which defines if a person gets more than US\$50k/year or not. 

The work reported in this notebook is part of a practice assignment in the discipline PCS5024 - Aprendizado Estatístico. This notebook uses the [Kedro](https://kedro.readthedocs.io/en/stable/) framework in orther to better organize data and project dependencies. 

In [1]:
train_data = catalog.load("adult_train_data")
test_data = catalog.load("adult_test_data")

2021-05-12 11:11:42,221 - kedro.io.data_catalog - INFO - Loading data from `adult_train_data` (CSVDataSet)...
2021-05-12 11:11:42,369 - kedro.io.data_catalog - INFO - Loading data from `adult_test_data` (CSVDataSet)...


/home/user01/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def prepare_adult_data(train_data, test_data):
    
    # Removendo 'missing data'
    train_data = train_data.dropna()
    
    